In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
# !git clone https://github.com/plutasnyy/uwmadison.git

In [56]:
# import sys
# sys.path.append('/kaggle/working/uwmadison')

In [57]:
from src.data_processing.constants import (
    KAGGLE_DATA_PATH,
    KAGGLE_OUTPUT_DATA_PATH,
    LOCAL_DATA_PATH,
    LOCAL_OUTPUT_DATA_PATH,
)

IS_KAGGLE = False
DATA_DIR = KAGGLE_DATA_PATH if IS_KAGGLE else LOCAL_DATA_PATH
OUTPUT_DIR = KAGGLE_OUTPUT_DATA_PATH if IS_KAGGLE else LOCAL_OUTPUT_DATA_PATH

In [ ]:
from src.data_processing.data_utils import prepare_df
from tqdm import tqdm

tqdm.pandas()

new_df = prepare_df(data_directory=DATA_DIR)
new_df.head()

In [ ]:
from src.data_processing.data_utils import load_img, id2mask, show_img
from matplotlib import pyplot as plt

row = 1
col = 4
plt.figure(figsize=(5 * col, 5 * row))
random_slices_to_plot = (
    new_df[~new_df.segmentation.isna()].sample(frac=1.0)["id"].unique()[: row * col]
)
for i, id_ in enumerate(random_slices_to_plot):
    img = load_img(new_df[new_df["id"] == id_].image_path.iloc[0])
    mask = id2mask(id_, df=new_df) * 255
    plt.subplot(row, col, i + 1)
    show_img(img, mask=mask)
    plt.tight_layout()

In [ ]:
from src.data_processing.data_utils import save_mask

save_mask(id_="case123_day22_slice_0117", df=new_df, save_dir=OUTPUT_DIR)

In [ ]:
import numpy as np

img = load_img(new_df[new_df["id"] == "case123_day22_slice_0117"].iloc[0].image_path)
mask_path = OUTPUT_DIR / "np" / "case123_day22_slice_0117.npy"
print(mask_path)
mask = np.load(str(mask_path))
plt.figure(figsize=(5, 5))
show_img(img, mask)

In [ ]:
ids = new_df["id"].unique()
save_mask(ids[5], new_df)

In [ ]:
from tqdm import tqdm
from joblib import delayed, Parallel

ids = new_df["id"].unique()
_ = Parallel(n_jobs=-1, backend="threading")(
    delayed(save_mask)(id_, new_df, OUTPUT_DIR) for id_ in tqdm(ids, total=len(ids))
)

In [ ]:
new_df["mask_path_png"] = str(OUTPUT_DIR) + "/png/" + new_df.id + ".png"
new_df["mask_path_np"] = str(OUTPUT_DIR) + "/np/" + new_df.id + ".npy"

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv("new_train", index=False)

In [ ]:
import shutil

if IS_KAGGLE:
    shutil.make_archive("/kaggle/working/png", "zip", str(OUTPUT_DIR / "png"))

In [ ]:
import shutil

if IS_KAGGLE:
    shutil.make_archive("/kaggle/working/np", "zip", str(OUTPUT_DIR / "np"))

In [ ]:
new_df["mask_path_png"] = str(OUTPUT_DIR) + "/png/" + new_df.id + ".png"
new_df["mask_path_np"] = str(OUTPUT_DIR) + "/np/" + new_df.id + ".npy"

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv("new_train.csv", index=False)

In [ ]:
import shutil

if IS_KAGGLE:
    shutil.make_archive("/kaggle/working/png", "zip", str(OUTPUT_DIR / "png"))

In [ ]:
import shutil

if IS_KAGGLE:
    shutil.make_archive("/kaggle/working/np", "zip", str(OUTPUT_DIR / "np"))

In [ ]:
!mkdir uw-madison-gi-tract-image-segmentation

In [ ]:
%cd /tmp

In [ ]:
!ls

In [ ]:
%cd data_processed

In [ ]:
!ls

In [ ]:
%cd png

In [ ]:
!du -sh png.zip

In [ ]:
%cd ..

In [ ]:
!du -sh np

In [21]:
new_df["mask_path_png"] = str(OUTPUT_DIR) + "/png/" + new_df.id + ".png"
new_df["mask_path_np"] = str(OUTPUT_DIR) + "/np/" + new_df.id + ".npy"

In [22]:
new_df.head()

In [28]:
new_df.to_csv("new_train.csv", index=False)

In [54]:
import shutil

if IS_KAGGLE:
    shutil.make_archive("/kaggle/working/png", "zip", str(OUTPUT_DIR / "png"))

In [63]:
import shutil

if IS_KAGGLE:
    shutil.make_archive("/kaggle/working/np", "zip", str(OUTPUT_DIR / "np"))

In [25]:
!mkdir uw-madison-gi-tract-image-segmentation

In [29]:
%cd /tmp

In [30]:
!ls

In [31]:
%cd data_processed

In [32]:
!ls

In [33]:
%cd png

In [62]:
!du -sh png.zip

In [35]:
%cd ..

In [38]:
!du -sh np